# <center>Supplementary material: calculating LD coefficients with PLINK</center>

This notebook contains the code for the download of data from the 1000 Genomes Project and the computation of linkage disequilibrium coefficients with the software PLINK.

First, we install data in the .vcf format from the 1000 Genomes Project for the three genes we are interested in (HLA-DRB1, TP53 and ABO), and save them in the "vcf_files" directory

In [ ]:
dir.create("vcf_files", showWarnings = FALSE)
system2(command = "wget", args = c("-c", "https://hgdownload.soe.ucsc.edu/gbdb/hg19/1000Genomes/phase3/ALL.chr6.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz"
, "-P", "./vcf_files"))
system2(command = "wget", args = c("-c", "https://hgdownload.soe.ucsc.edu/gbdb/hg19/1000Genomes/phase3/ALL.chr6.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz.tbi"
, "-P", "./vcf_files"))

system2(command = "wget", args = c("-c", "https://hgdownload.soe.ucsc.edu/gbdb/hg19/1000Genomes/phase3/ALL.chr9.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz"
, "-P", "./vcf_files"))
system2(command = "wget", args = c("-c", "https://hgdownload.soe.ucsc.edu/gbdb/hg19/1000Genomes/phase3/ALL.chr9.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz.tbi"
, "-P", "./vcf_files"))

system2(command = "wget", args = c("-c", "https://hgdownload.soe.ucsc.edu/gbdb/hg19/1000Genomes/phase3/ALL.chr17.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz"
, "-P", "./vcf_files"))
system2(command = "wget", args = c("-c", "https://hgdownload.soe.ucsc.edu/gbdb/hg19/1000Genomes/phase3/ALL.chr17.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz.tbi"
, "-P", "./vcf_files"))

system2(command="wget", args=c("https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel"
, "-P", "./vcf_files")) #this is the .panel file, useful to separate data for different populations

Now we install the version 1.9 of the PLINK software.

In [10]:
url <- "https://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20231211.zip"
dest <- "plink.zip"

download.file(url, dest)
unzip(dest, exdir = "plink")

Sys.chmod("plink/plink", mode = "0755")

We set some parameters to navigate into the 1000 Genomes Project dataset. We want data from the three genes with the starting and ending positions given by the first and last variant in the gnomAD dataset for each gene. We keep only data about African population, but we still use a vector of populations so that it is easier to add other populations if we need.

In [19]:
genes <- data.frame( #data frame with the genes
  gene  = c("HLA-DRB1","TP53","ABO"),
  chromosome   = c("6", "17", "9"),
  start = c(32579019, 7661882, 133233496), #start and stop taken from gnomAD dataset
  end   = c(32589816, 7676669, 133275264),
  stringsAsFactors = FALSE
)

populations<-c("AFR") #only Africans, but we can add other populations if we want
command<-"./plink/plink" #the command to use PLINK, useful later

Now we write a .keep file: it will be useful later to keep only data about African population.

In [ ]:
panel <- read.delim(
  "vcf_files/integrated_call_samples_v3.20130502.ALL.panel",
  stringsAsFactors = FALSE
) #read the .panel file

write_keep <- function(superpop, out_file) {
  ids <- panel$sample[panel$super_pop == superpop]
  keep <- data.frame(FID = ids, IID = ids)
  write.table(keep, out_file,
              quote = FALSE, row.names = FALSE,
              col.names = FALSE, sep = "\t")
}

write_keep("AFR", "AFR.keep")

Using PLINK, we extract data about the three genes in African population, keeping only SNPs for positions in which we have no more than one variant, and save the results in the "plink_out" folder.

In [ ]:
dir.create("plink_out", showWarnings = FALSE) #create a new folder with the results if it does not exist
for (i in 1:3){
    file_path<-sprintf("vcf_files/ALL.chr%s.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz", genes$chromosome[i])
    for (pop in populations){
        output_file_name<-sprintf("plink_out/%s_%s", genes$gene[i], pop) #name of the output file
        keep_file_name<-sprintf("%s.keep", pop) #file .keep for the population to keep
        args<-c("--vcf", file_path,
                "--keep", keep_file_name,
                "--chr", genes$chromosome[i],
                "--from-bp", genes$start[i],
                "--to-bp", genes$end[i],
                "--snps-only", "just-acgt", #only SNPs
                "--biallelic-only", "strict", #only biallelic (no more than one variant)
                "--make-bed",
                "--out", output_file_name)
        system2(command, args) #system2 executes a command as in the command line
    }
}

We compute the LD coefficients keeping only common variants and considering only couples of positions with distance no higher than 500 kb.

In [22]:
dir.create("linkage_disequilibrium", showWarnings = FALSE)
for (i in 1:3){
    for (pop in populations){
        input_file_name<-sprintf("plink_out/%s_%s", genes$gene[i], pop)
        output_file_name<-sprintf("linkage_disequilibrium/%s_%s_LD", genes$gene[i], pop)
        args<-c("--bfile", input_file_name,
                "--maf", "0.01", #only common variants (frequency higher than 1%)
                "--r2", "gz",
                "--ld-window-kb", "500",
                "--ld-window", "99999",
                "--ld-window-r2", "0", #keeps all the LD (no minimum threshold)
                "--out", output_file_name
       )
        system2(command, args)
    }
}

Finally, we create the .csv files associated to those data

In [25]:
library(tidyverse) #for the read_table function

dir.create("linkage_disequilibrium_csv", showWarnings = FALSE)
for (i in 1:3){
    for (pop in populations){
        input_file_name<-sprintf("linkage_disequilibrium/%s_%s_LD.ld.gz", genes$gene[i], pop)
        output_file_name<-sprintf("linkage_disequilibrium_csv/%s_%s_LD.csv", genes$gene[i], pop)
        table<-read_table(input_file_name)
        write.csv(table, output_file_name, row.names=FALSE)
    }
}

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
“Missing column names filled in: 'X8' [8]”

── Column specification ────────────────────────────────────────────────────────
cols(
  CHR_A = col_double(),
  BP_A = col_double(),
  SNP_A = col_character(),
  CHR_B = col_double(),
  BP_B = col_double(),
  SNP_B = col_character(),
  R2 = col_double(),
  X8 = col_logical()
)

Warning message:
“Missing column names filled in: 'X8' [8]”

── Column specification ──────────────────────────────────────────────────